# 1. Cleaning

Note: the existing "2008.csv" file in the directory could be the smaller version of the original.
Consider downloading full-size file from http://stat-computing.org/dataexpo/2009/2008.csv.bz2

In [ ]:
# import neccessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# show plot intermediately under the calling cell
%matplotlib inline

# disable some warnings
import warnings
warnings.simplefilter(action = "ignore")

# set to unlimited column display:
pd.set_option('display.max_columns', None)

# import database
df = pd.read_csv('2008.csv')

<center>Table: (sample) Dataframe including column dtype</center>

In [ ]:
dtypes = pd.Series(df.dtypes,name='dtypes')
df.head(2).append(dtypes)

## 1.1. Descriptive Statistics

<center>Figure: Correlation Matrix of some columns</center>

In [ ]:
def plot_corr_matr(df, figsize=(10, 8)):
    """
    Plot Correlation matrix using Seaborns heatmap

    Parameters
    ----------
    df: pandas.DataFrame

    figsize: tuple
        size of the whole plot
    """
    plt.figure(figsize=figsize)
    corr = df.corr().round(2)
    # mask = np.zeros_like(corr)
    # mask[np.triu_indices_from(mask)] = True
    sns.heatmap(
        corr,
        cmap="seismic",
        square=True,
        xticklabels=corr.columns.values,
        yticklabels=corr.columns.values,
        center=0,
        annot=True)
    plt.xticks(rotation=45)
    plt.show()
    
plot_corr_matr(df.loc[:,'DepTime':'Distance'])
# plot_corr_matr(df)

//////// _From the graph above, which features, in your opinions, should be removed, which should be kept, why?_

## 1st

-To classify whether a flight is late or not, choose __ArrTime as the label__.

-__ArrTime__, and __DepTime__ although have high correlation (0.72) but still not enough, we leave it to PCA to eliminate collinearity.

-__CRSArrTime__ and __ArrTime__ also has 0.87 correlation score, we just keep __ArrTime__ as it is the real time data.

-__ActualElapsedTime, CRSElapedTime, AirTime__ and __Distance__ have very high correlation (>0.95) so we can just keep one feature (Distance) as predictor.

-__CRSDepTime__ and __DepTime__ also has 0.97 correlation score, we just keep __DepTime__ as it is the real time data.

-__DepDelay__ is a predictor and __ArrDelay__ is a lable. __DepDelay__ and __ArrDelay__ have very high correlation (0.93).
So we want to see if other features can be a good predictors to ArrDelay, __delete DepDelay is the choice__.










In [ ]:
# delete CRSDepTime, CRSArrTime, ActualElapsedTime, CRSElapsedTime, AirTime, DepDelay
del df['CRSDepTime']
del df['CRSArrTime']
del df['ActualElapsedTim']
del df['CRSElapsedTime']
del df['AirTime']
del df['DepDelay']

In [ ]:
# drop columns:
df = df.loc[:,'DepTime':'Distance']

////////// End the 1st request

<center>Table: Describe database</center>

In [ ]:
df.describe(include='all').T

## 1.2. NaN Cleaning

<center>Table: Recheck describe columns with NaN values</center>

In [ ]:
from IPython.display import display
import pandas as pd
def desc_nan(df):
    """
    Show a table that describe about all columns of @df that contain NaN.
    """
    row, col = df.shape
    nas = row - df.describe(include='all').T['count']
    nas = nas[nas > 0]
    ps = nas / row * 100
    ms = df.mode().iloc[0]
    nadf = pd.DataFrame(data={
        'Na Count': nas,
        'Na Percentage': ps,
        'mode': ms
    }).loc[nas.index]
    dnadf = df.describe().T.reindex(nadf.index)
    nadf = pd.concat([nadf, dnadf.iloc[:, 1:]], axis=1)
    display(nadf)

desc_nan(df)

<center>Figure: Distribution plot of all numeric columns</center>

In [ ]:
def plot_ft_dist(df, fig_height=4, no_of_col=2,verbose=True):
    """
    Plot distribution of all numeric columns in df

    Parameters
    ----------
    df: pandas.DataFrame

    fig_height: int

    no_of_col: int
    """
    df = df.copy()
    # prepare df
    cat_cols = list(df.dtypes[df.dtypes == 'object'].index)
    if verbose:
        print("Ignored categorical columns: ", cat_cols)
        print("")
    df_hasnan = df.isna().any().any()
    if df_hasnan:
        nan_cols = list(df.loc[:,df.isna().any()].columns)
        df.dropna(inplace=True, axis=1)
        if verbose:
            print("dropped NaN cols:", str(nan_cols))
            print("")
    try:    
        desc = df.describe().T
        std0 = list(desc.loc[desc['std']==0].index)
        if verbose:
            print("dropped std=0 cols:", str(std0))
            print("")
        for col in std0:
            del df[col]
    except KeyError:
        pass
    
    idx = df.dtypes[df.dtypes != 'object'].index
    # prepare frame
    f, axes = plt.subplots(
        int(np.ceil(len(idx) / no_of_col)),
        no_of_col,
        figsize=(5 * no_of_col, np.ceil(len(idx) / no_of_col) * fig_height))
    sns.set(style="white", palette="muted", color_codes=True)
    n = 0
    for i in idx:
        sns.distplot(
            df[i],
            color='b',
            hist=True,
            kde_kws={"shade": True},
            ax=axes[n // no_of_col, n % no_of_col])
        n += 1
    plt.show()

plot_ft_dist(df)

////// _Run codes above, describe what you see and start clean data / remove unimportant columns._

## 2nd 


In [ ]:
# delete CRSDepTime, CRSArrTime, ActualElapsedTime, CRSElapsedTime, AirTime, DepDelay
del df['CRSDepTime']
del df['CRSArrTime']
del df['ActualElapsedTim']
del df['CRSElapsedTime']
del df['AirTime']
del df['DepDelay']

In [ ]:
# drop columns:
df = df.loc[:,'DepTime':'Distance']

In [ ]:
df.describe(include='all').T

////////// End the 2nd request

<center>Table: (sample) Database after cleaning</center>

In [ ]:
df.head(2)

## 1.3. Feature Engineering

/////////// Give some insights, split train-test dataset




### 1.3.1 Data Transformation

__DepTime, ArrTime__ are time values which if being stored in 10-based (decimal), their true values will be distorted (hours:minutes is not continuous on 10-based).
#=> convert to minute integer

In [ ]:
# convert column to min:
def to_min(n):
    n = str(int(n))
    if len(n)<=2:
        return int(n)
    else:
        return int(n[:-2])*60+int(n[2:])
df.DepTime=df.DepTime.apply(to_min)
df.ArrTime=df.ArrTime.apply(to_min)

Convert all float dtype columns to int64 dtype

In [ ]:
for col in df.columns:
    if df[col].dtype == np.float:
        df[col] = df[col].astype(np.int64)

<center>Table: (sample) Dataframe after dtype convertion</center>

In [ ]:
dtypes = pd.Series(df.dtypes,name='dtypes')
df.head(2).append(dtypes)

### 1.3.2. One-hot-encoding

In order to pick which feature to use One-hot-encoding, we see how many unique data in each category data, then use boxplot to check the variation of ArrDelay by those features.
<br><br>
<center>Table: Unique value from each categorial feature</center>

In [ ]:
pd.DataFrame(df.describe(include="O").loc['unique']).T

/////// _start your code below to one hot encode or label encoding categorical features_

## 3rd




In [ ]:
sns.set(style="white")
f, ((ax1, ax2),(ax3, ax4)) = plt.subplots(2,2,sharex=True, squeeze=True, 
                                figsize=(10,10))

sns.boxplot(x="ArrDelay", y="UniqueCarrier", data=df,
            whis=np.inf, palette="vlag",ax=ax1)

sns.boxplot(x="ArrDelay", y="TailNum", data=df,
            whis=np.inf, palette="vlag",ax=ax2)

sns.boxplot(x="ArrDelay", y="Origin", data=df,
            whis=np.inf, palette="vlag",ax=ax3)

sns.boxplot(x="ArrDelay", y="Dest", data=df,
            whis=np.inf, palette="vlag", ax=ax4)
plt.show()

We assume that each __TailNum__ belongs to 1 __UniqueCarrier__. How late a flight is seems to more dependent on __UniqueCarrier__ than the __TailNum__. But we need to check this hypothesis by showing if there is any __TailNum__ that is NOT operated by 1 __UniqueCarrier__.
<br><br>
<center>Table: TailNum that is not operated by one UniqueCarrie</center>

In [ ]:
# Check if there is any TailNum that belongs to 2 UniqueCarriers
pd.DataFrame(df.groupby('TailNum').UniqueCarrier.unique()
                [df.groupby('TailNum').UniqueCarrier.unique()
                     .apply(lambda x: len(x)!=1)])

The table rejects our hypothesis, but it shows only 2 instances of __TailNum__.
The total number of records of N376CA and N379CA is 3171/6855024 (0.0462%) which is a very small affect (if any) on the prediction result. In addition, the amount of unique __TailNum__ is too large (5366) that might result in not having enough samples for each unique __TailNum__ (which makes the result more unreliable)

__Origin__ or __Dest__ can be reliable predictors but currently we skip these rows for computational benefits.

After finish analyzing those 4 categorial features, only 1 column to one-hot-encode is __UniqueCarrier__.

In [ ]:
# one-hot-encode UniqueCarrier column
carrier = pd.get_dummies(df.UniqueCarrier)
carrier.reset_index(drop=True, inplace=True)

////////// End the 3rd request

## 2.2. Normalize data

/////// _normalize / scale your data to range [0,1]

## 4th



Because of feature scaling, new table of normalized data (in range [0,1]) is needed.

In [ ]:
# delete categorical data:
for col in df.columns:
    if df[col].dtype == np.object:
        del df[col]

# Normalize the df
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(df)
X = pd.DataFrame(X, columns=df.columns)

In [ ]:
# combine normalized data with one-hot-encoded columns
X = X.join(carrier)

In [ ]:
# assign ArrDelay in format of boolean, True if ArrDelay > 30 minutes
Y = df.ArrDelay>30
Y.reset_index(drop=True, inplace=True)
del df['ArrDelay']

////////// End the 4th request

<center>Table: (sample) Predictor X (without PCA)

In [ ]:
X[:2]

## 2.3. PCA

<center>Figure: Explaination ratio though dimensions using PCA and Cumulative explained variance table in details</center>

In [ ]:
def plot_PCA(df):
    """
    Show the "PCA ladder" in order to pick the right

    Parameters
    ----------
    df: pandas.DataFrame

    fig_height: int

    no_of_col: int
    """
    from sklearn.decomposition import PCA

    pca = PCA(n_components=len(df.columns))
    pca.fit_transform(df)
    # put data (predictor only) to pca and create pcadf
    ratio = pca.explained_variance_ratio_

    tot = sum(ratio)
    var_exp = [(i / tot) for i in ratio]
    cum_var_exp = np.cumsum(var_exp)

    plt.bar(
        range(0, len(df.columns)),
        var_exp,
        alpha=0.5,
        align='center',
        label='individual explained variance')
    plt.step(
        range(0, len(df.columns)),
        cum_var_exp,
        where='mid',
        label='cumulative expalined variance')
    plt.ylabel = 'Explained variance ratio'
    plt.xlabel = 'Pricipal components'
    plt.legend(loc='best')
    plt.show()

    # show table with more details
    pdf = pd.DataFrame(
        [cum_var_exp],
        columns=["dim {}".format(i) for i in range(1,
                                                   len(df.columns) + 1)])
    display(pdf)
    
    
plot_PCA(X)

//////////// the code below has "n" as dimensions to reduce to, decide that number so the total explation ratio > 0.95

## 5th




In [ ]:
from sklearn.decomposition import PCA
# create a PCA to reduce dimension to n
n = # ?????
pca = PCA(n_components=n)
# put data (predictor only) to pca and create pcadf
X = pd.DataFrame(pca.fit_transform(X))

In [ ]:
# get random 100_000 samples records from full dataset to tune
x_tune = X.sample(n=100_000, random_state=25)
y_tune = Y.sample(n=100_000, random_state=25)

# get random 1_250_000 samples records from full dataset
# that differ from the x_100k and y_100k
x = (X[~X.index.isin(x_tune.index)]
        .sample(n=1_250_000, random_state=25))
y = (Y[~Y.index.isin(x_tune.index)]
        .sample(n=1_250_000, random_state=25))

In [ ]:
# prepare the train and test data.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
                        x, y, test_size=0.2, random_state=25)

//////// End the 5th request

In [ ]:
# Save processed database to a csv file
import os
os.chdir(r'.\working_database')

x_tune.to_csv('x_tune.csv')
pd.DataFrame(y_tune).to_csv('y_tune.csv')
x_train.to_csv('x_train.csv')
pd.DataFrame(y_train).to_csv('y_train.csv')
x_test.to_csv('x_test.csv')
pd.DataFrame(y_test).to_csv('y_test.csv')

os.chdir('..')

# 3. Classification Models

In [ ]:
# import neccessary libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# show plot intermediately under the calling cell
%matplotlib inline

# disable some warnings
import warnings
warnings.simplefilter(action = "ignore")

# set to unlimited column display:
pd.set_option('display.max_columns', None)

# Load processed database from a csv file
import os
os.chdir(r'.\working_database')

x_tune = pd.read_csv('x_tune.csv', index_col=0)
y_tune = pd.read_csv('y_tune.csv', index_col=0)
x_train = pd.read_csv('x_train.csv', index_col=0)
y_train = pd.read_csv('y_train.csv', index_col=0)
x_test = pd.read_csv('x_test.csv', index_col=0)
y_test = pd.read_csv('y_test.csv', index_col=0)

os.chdir('..')

The following model will use default parameters and sample classification report

## 3.1. Naive Bayes

Initialize the model and fit the train database

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
print('Classification_report: \n', 
      classification_report(
          y_true=y_test,
          y_pred=nb.predict(x_test)))

////// Search google for "sklearn logistics regression" / decision tree / svm and figure out how to work on below parts

## 6th



## 3.2. Logistics Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2', # 'l1'
                        C = 1.0) # float

In [ ]:
from sklearn.model_selection import GridSearchCV
# Defind the parameter values that should be searched
C_list = [0.1, 1, 10, 100, 1000]
penalty_list = ['l1','l2']
solver_list = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

# Create a parameter grid: map the parameter names to the
# values that should be searched:
param_grid = dict(C=C_list,
                 penalty=penalty_list,
                 solver=solver_list)
# Instantiate the grid:
grid = GridSearchCV(lr, param_grid, cv=10, error_score=0.0,
                    scoring='f1_weighted', n_jobs=16)

# fit the grid with data
grid.fit(x_tune, y_tune)

# examine the best model
print(grid.best_score_)
print(grid.best_params_)

# gre = Gridsearchcv REsult
gre = pd.DataFrame(grid.cv_results_)
(gre.loc[gre.mean_test_score!=0] # skip zero scored rows
     [['rank_test_score','mean_fit_time','mean_test_score'] # get these col
          +['param_{}'.format(i) for i in grid.best_params_.keys()]]
    .sort_values('rank_test_score')) # sort by rank

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=0.1,
                        penalty='l1',
                        solver='liblinear')
lr.fit(x_train, y_train)

from sklearn.metrics import classification_report
print('Classification_report: \n', 
      classification_report(
          y_true=y_test,
          y_pred=lr.predict(x_test)))

## 3.3. Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='gini', # 'entropy'
                             min_samples_leaf=1, # float
                             class_weight=None # 'balance'
                             ) 
# tree.fit(x_train, y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV
# Defind the parameter values that should be searched
criterion_list = ['gini','entropy']
min_samples_leaf_list = [1,2,4,6,8,10]
class_weight_list = [None,'balanced']
# Create a parameter grid: map the parameter names to the
# values that should be searched:
param_grid = dict(criterion=criterion_list,
                 min_samples_leaf=min_samples_leaf_list,
                 class_weight=class_weight_list)

# Instantiate the grid:
grid = GridSearchCV(tree, param_grid, cv=10, 
                    scoring='f1_weighted', n_jobs=16)

# fit the grid with data
grid.fit(x_tune, y_tune)

# examine the best model
print(grid.best_score_)
print(grid.best_params_)

# gre = Gridsearchcv REsult
gre = pd.DataFrame(grid.cv_results_)
(gre.loc[gre.mean_test_score!=0] # skip zero scored rows
     [['rank_test_score','mean_fit_time','mean_test_score'] # get these col
          +['param_{}'.format(i) for i in grid.best_params_.keys()]]
    .sort_values('rank_test_score')) # sort by rank

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(criterion='entropy',
                              min_samples_leaf=2,
                              class_weight=None)
                              
tree.fit(x_train, y_train)

from sklearn.metrics import classification_report
print('Classification_report: \n', 
      classification_report(
          y_true=y_test,
          y_pred=nb.predict(x_test)))

## 3.4. SVM

In [ ]:
from sklearn.svm import SVC
svm = SVC(C = 1.0,       # float
         kernel = 'rbf', # 'linear','poly', 'sigmoid'
         gamma = 'auto') # float
# svm.fit(x_train, y_train)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Defind the parameter values that should be searched
C_list = [1,10,100,1000]
kernel_list = ['rbf','linear','poly','sigmoid']
gamma_list = ['auto', 10, 0.2]
# Create a parameter grid: map the parameter names to the
# values that should be searched:
param_grid = dict(C=C_list,
                  kernel=kernel_list,
                  gamma=gamma_list)

# Instantiate the rand:
rand = RandomizedSearchCV(svm, param_grid, cv=5, 
                         scoring='f1_weighted', n_jobs=16,
                         n_iter=5, random_state=25)

# fit the rand with data
rand.fit(x_tune, y_tune)

# examine the best model
print(rand.best_score_)
print(rand.best_params_)

# gre = Gridsearchcv REsult
gre = pd.DataFrame(rand.cv_results_)
(gre.loc[gre.mean_test_score!=0] # skip zero scored rows
     [['rank_test_score','mean_fit_time','mean_test_score'] # get these col
          +['param_{}'.format(i) for i in rand.best_params_.keys()]]
    .sort_values('rank_test_score')) # sort by rank

In [ ]:
from sklearn.svm import SVC
svm = SVC(kernel='rbf')               
svm.fit(x_train, y_train)

from sklearn.metrics import classification_report
print('Classification_report: \n', 
      classification_report(
          y_true=y_test,
          y_pred=svm.predict(x_test)))

////////// End the 6th request

# Z. Other
Additional parts

## Z.1. Pickle

In [ ]:
# Save a model into a pickle file.
import pickle
import os
os.chdir(r'.\saved_models')
pickle.dump(svm, open( "svm.model", "wb" ) )
os.chdir('..')

In [ ]:
# Load the model back from the pickle file.
import pickle
import os
os.chdir(r'.\saved_models')
lr = pickle.load(open("lr.model", "rb"))
nb = pickle.load(open("nb.model", "rb"))
rf = pickle.load(open("rf.model", "rb"))
tree = pickle.load(open("tree.model", "rb"))
svm = pickle.load(open("svm.model", "rb"))
os.chdir('..')